In [2]:
import whisper
import torch
import os
os.listdir()

torch.cuda.is_available()
torch.cuda.empty_cache()


In [3]:
model = whisper.load_model("medium.en", device="cuda:0")
audio = whisper.load_audio('test2.mp3')

result = model.transcribe(audio)
print(result["text"])

 Sorry, that was a restaurant thing. I was going to just ask who did you receive the funds for? She's always just a possible for... She's just funding it in communications. So, lots of people reach out to her and she gets in touch with me. People reach out to her in some areas that are... They're seeing more of the other people, like needle debris or something like that. So she'll reach out to me and I'll go kind of suit the area and keep on it for a little bit. See how that's... how frequent. I wonder if it's like a citizen or if it's like businesses in the area. Because that's like, I guess, what would come to mind for me. Yeah, it's pretty even right now. Yeah, it's through the calls and there's control over it. How pretty is it? It seems like a good mixture now that it's warm out of here. Yeah, it's like if I could maybe... Yeah, you would know. Yeah, I guess you'd have to break it up. Well, Excel spreadsheet. Yeah, she does. How frequently do you get calls from this location? I th

In [4]:
from ctransformers import AutoModelForCausalLM, AutoConfig

config = AutoConfig.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF")
# Explicitly set the max_seq_len
config.max_seq_len = 4096
config.max_answer_len= 4096

# Set gpu_layers to the number of layers to offload to GPU. 
# Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True,
    config=config,
    device_map="auto"
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# from transformers import AutoModelForCausalLM

# # Set gpu_layers to the number of layers to offload to GPU. 
# # Set to 0 if no GPU acceleration is available on your system.
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
#                                              gpu_layers=50)

In [5]:
from transformers import AutoTokenizer, pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
torch.cuda.empty_cache()

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Pipeline
generator = HuggingFacePipeline(
    pipeline=
        pipeline(
            model=model, 
            tokenizer=tokenizer, 
            config=config,
            task='text-generation',
            max_new_tokens = 100,
            device_map='auto'),
    batch_size=10)


In [6]:
#Verifying LLM
response = generator("What is 1+1?")
response
print(response)


A: 2


In [7]:
#Truncating length
print(len(result["text"]))

if len(result["text"]) / 4 > 256:
    print("too long")
    result["text"] = result["text"][1:512]
else:
    print("OK!")

print(len(result["text"]))
print(result["text"])

1742
too long
511
Sorry, that was a restaurant thing. I was going to just ask who did you receive the funds for? She's always just a possible for... She's just funding it in communications. So, lots of people reach out to her and she gets in touch with me. People reach out to her in some areas that are... They're seeing more of the other people, like needle debris or something like that. So she'll reach out to me and I'll go kind of suit the area and keep on it for a little bit. See how that's... how frequent. I wonder if i


In [8]:
#Prompting
prompt = f"""
Return any short phrases (4-5 words) that refer to a space, location or place from the following document in a bulleted list. Do not return anything else as part of your answer.
Here is the following document:
\"{result["text"]}\"


"""
response = generator(prompt)
print(response)

* Restaurant
* Funds
* Communications
* Needle debris
* Area
* Suit
* Keep on it
* Frequency
* Wonder
* I
